In [1]:
# In[]
from random import randint
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
import numpy as np
# In[]


import string
import re
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token


    tokens = [re.sub(r'[^\w\s]','',w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens


# In[]
from nltk.corpus import gutenberg

tokens_raw1 = gutenberg.words('austen-emma.txt')
n1 = len(tokens_raw1)
tokens_train1  = tokens_raw1[0:int(0.8*n1)];
tokens_test1 = tokens_raw1[int(0.8*n1):n1]; 

tokens_raw2 = gutenberg.words('austen-sense.txt')
n2 = len(tokens_raw2)
tokens_train2  = tokens_raw2[0:int(0.8*n2)];
tokens_test2 = tokens_raw2[int(0.8*n2):n2]; 

tokens_raw = tokens_train1 + tokens_train2
tokens_test = tokens_test1 + tokens_test2

text_raw = ' '.join(tokens_raw)
tokens = clean_doc(text_raw)



len(tokens_test)


/home/chaikesh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


66802

In [2]:
# In[]
# organize into sequences of tokens
length = 50 + 1
step =2
sequences = list()
for i in range(0,len(tokens)-length, step):
	# select sequence of tokens
	seq = tokens[i:i+length]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 112954


In [3]:
lines = sequences
# In[]
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

# In[]

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# In[]
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
seq_length

50

In [4]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 500, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 500)           4139000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           240400    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 8278)              836078    
Total params: 5,305,978
Trainable params: 5,305,978
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
# In[]

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=80)

# In[]

# save the model to file
model.save('model_a03_wl_500_100_80.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer_a03_wl_500_100_80.pkl', 'wb'))

Epoch 1/80
112954/112954 [==============================] - 401s 4ms/step - loss: 6.3789 - acc: 0.0387
Epoch 2/80
112954/112954 [==============================] - 428s 4ms/step - loss: 5.9430 - acc: 0.0642
Epoch 3/80
112954/112954 [==============================] - 439s 4ms/step - loss: 5.7418 - acc: 0.0800
Epoch 4/80
112954/112954 [==============================] - 438s 4ms/step - loss: 5.5589 - acc: 0.0961
Epoch 5/80
112954/112954 [==============================] - 439s 4ms/step - loss: 5.3702 - acc: 0.1100
Epoch 6/80
112954/112954 [==============================] - 435s 4ms/step - loss: 5.2284 - acc: 0.1214
Epoch 7/80
112954/112954 [==============================] - 450s 4ms/step - loss: 5.1126 - acc: 0.1301
Epoch 8/80
112954/112954 [==============================] - 454s 4ms/step - loss: 5.0072 - acc: 0.1383
Epoch 9/80
112954/112954 [==============================] - 450s 4ms/step - loss: 4.9113 - acc: 0.1443
Epoch 10/80
112954/112954 [==============================] - 445s 4ms/ste

In [5]:
from random import randint
from pickle import load
from keras.models import load_model
# load the model
model = load_model('model_a03_wl_500_100_80.h5')
# load the tokenizer
tokenizer = load(open('tokenizer_a03_wl_500_100_80.pkl', 'rb'))

In [6]:
# In[]
from keras.preprocessing.sequence import pad_sequences


# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
        #yhat1 = model.predict(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [9]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
generated = generate_seq(model, tokenizer, seq_length, seed_text, 15)
print(generated)


mourning in secret over obstacles which must divide her for ever from the object of her love and that marianne was internally dwelling on the perfections of a man of whose whole heart she felt thoroughly possessed and whom she expected to see in every carriage which drove near their house

and of temptation her own imagination and impetuous him that she knew her impatience to


In [10]:
#perplexity

test_tokens = tokens_test[1:10000]

in_text = test_tokens[0]
logp = 0
c=0
for word in test_tokens:
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    yhat1 = model.predict(encoded,verbose=0)[0]
    next_word = word
    if next_word in tokenizer.word_index:
        p = yhat1[tokenizer.word_index[next_word]]
        if  p >=0.00001:
            c+=1
            logp = logp + np.log(p)

    else:
        p = 1
        c+=1
        logp = logp + np.log(p)
    
    in_text +=' ' + next_word
pplxty = np.exp(-1*logp/c)

In [11]:
print c
pplxty


7924


31.732750104716793